# Import dependencies

In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

%matplotlib inline

# Functions

In [ ]:
def plot_region(df, regione):
    df_regione = df[df["denominazione_regione"] == regione]
    province = list(df_regione["denominazione_provincia"].unique())
    
    plt.rcParams.update({"legend.fontsize": "x-large",
                         "figure.figsize": (20, 15),
                         'xtick.labelsize': 10})
    ax = plt.gca()
    formatter = mdates.DateFormatter("%m-%d")
    ax.xaxis.set_major_formatter(formatter)
    locator = mdates.DayLocator()
    ax.xaxis.set_major_locator(locator)

    for i in range(len(province)):
        df_provincia = df_regione[df_regione["denominazione_provincia"] == province[i]]
        
        data = df_provincia["data"].values
        totale_casi = df_provincia["totale_casi"].values
        plt.plot(data, totale_casi, label=province[i])
    
    totale_casi = (df[df["denominazione_regione"] == regione]
                   .groupby(["data"])["totale_casi"]
                   .sum().values)
    plt.plot(data, totale_casi, label="Totale %s" % regione)
    
    plt.legend(loc="upper left")
    plt.show()    

# Load data

In [ ]:
dati_province_fpath = "../../dati-province/dpc-covid19-ita-province.csv"
df = pd.read_csv(dati_province_fpath, sep=",")

In [ ]:
df.head()

## Coverti data: string to datetime object

In [ ]:
df.loc[:, "data"] = df.loc[:, "data"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [ ]:
df.head()

# Totale per regione

In [ ]:
df_totale_regione = (df[ df["data"] == df["data"].max() ]
                     .groupby(["denominazione_regione"], as_index=False)["totale_casi"].sum()
                     .sort_values(["totale_casi"], ascending=[False])) 

In [ ]:
df_totale_regione

In [ ]:
df_totale_regione["totale_casi"].sum()

## Senza Lombardia

In [ ]:
df_totale_regione = (df[ (df["data"] == df["data"].max()) & 
                         (df["denominazione_regione"] != "Lombardia") ]
                     .groupby(["denominazione_regione"], as_index=False)["totale_casi"].sum()
                     .sort_values(["totale_casi"], ascending=[False])) 

In [ ]:
df_totale_regione["totale_casi"].sum()

# Andamento Lombardia

In [ ]:
plot_region(df, "Lombardia")

# Andamento Emilia Romagna

In [ ]:
plot_region(df, "Emilia Romagna")

# Andamento Veneto

In [ ]:
plot_region(df, "Veneto")

# Andamento Sardegna

In [ ]:
df_province = (df[(df["data"] == df["data"].max()) & 
         (df["denominazione_regione"] == "Sardegna")][["denominazione_provincia", "totale_casi"]]
     )

In [ ]:
df_province

In [ ]:
plot_region(df, "Sardegna")